Signal 6 : Timeo Martin, Arno Braz, Julie Le Ral

# I/ Lecture et Affichage

In [ ]:
import numpy as np
import msicpe.san as san
from plotly import express as px
import scipy.signal as signal

signal_dict = np.load('../Sons/apprentissage/a_1.npz') # signal_dict est un dictionnaire → contenant les données contenues dans le fichier signal.npz
s = signal_dict['s'] # extraction du signal utile
t = signal_dict['t'] # extraction du vecteur temps associé au signal
trace_signal = px.line(x=t, y=s, labels={'x': 'Temps (t)', 'y': 's'})
tf = san.trans_fourier(s, t)
trace_spectre = px.line(x=tf[0], y=abs(tf[1]), labels={'x': 'frequences (nu)', 'y': 'S'})
trace_log10_spectre = px.line(x=tf[0], y=np.log10(abs(tf[1])), labels={'x': 'frequences (nu)', 'y': 'S'})
trace_signal.show()
trace_spectre.show()
trace_log10_spectre.show()


FileNotFoundError: [Errno 2] No such file or directory: 'apprentissage/a_1.npz'

T = (0.5-0.36)/15 = 0.00933s  
nu0 = 107.2Hz -> Correspond à l'oObservation graphique  
C'est plus lisible en échelle log10 et donc plus adapté (notament pour les hautes fréquences) pour lire les harmoniques distantes de la fondamentale


# /II Apprentissage

In [ ]:
H = 15 #nbr d'harmoniques

dict_i = dict()

for n in range(1, 6):
    signal_dict = np.load(f'../Sons/apprentissage/i_{n}.npz') # signal_dict est un dictionnaire → contenant les données contenues dans le fichier signal.npz
    s_i = signal_dict['s'] # extraction du signal utile
    t_i = signal_dict['t'] # extraction du vecteur temps associé au signal

    nu_i, S_i = tf_i = san.trans_fourier(s_i, t_i)

    nu0_i = san.detect_fondamentale(np.abs(S_i), nu_i, threshold=100) #float, treshold : valeur arbitraire
    harm_i = san.detect_pics(S_i, [k*nu0_i for k in range(1, H+1)], nu_i) #array

    dict_i[f'i_{n}'] = [nu0_i, harm_i]


print(nu0_i, harm_i)


137.95400383333333 (array([ 137.977     ,  276.95383333,  413.931     ,  551.908     ,
        688.88516667,  826.86216667,  963.83933333, 1094.8175    ,
       1242.79283333, 1371.77133333, 1502.7495    , 1653.72433333,
       1787.702     , 1924.67916667, 2053.65766667]), array([9.3468335e+02-5.8716595e+02j, 2.0984425e+02-1.2211643e+02j,
       5.5669353e+01+2.3755293e+00j, 1.6448006e+01-1.7990946e+01j,
       5.4189014e+00-9.2099333e-01j, 3.9129484e+00-5.0125394e+00j,
       1.5686276e+00-1.4222233e-01j, 1.2927541e+00-9.6247381e-01j,
       8.6327285e-01-6.9309884e-01j, 9.1000420e-01-8.5097057e-01j,
       9.4015718e-01-1.0093188e+00j, 1.1661974e+00-8.3360016e-01j,
       1.7652042e+00-2.6831132e-01j, 1.9106065e+00-3.9666271e-01j,
       7.5696468e+00-1.2792455e-01j], dtype=complex64))
